In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 구글 드라이브 전체 설치 (최초 한번만 설치)
#@markdown <div><font color="red">5GB~7GB정도 필요해요. 구글드라이브 용량을 확인해주세요</font></div>  
#@markdown * 설치후에 <a href="https://github.com/ninjaneural/webui/blob/master/memo/install.md">이곳</a>에서 실행방법을 확인해주세요*
Google_Install = True

#@markdown **WebUI Version**
TagVersion = 'v1.7.0' #@param ["v1.6.0", "v1.7.0"]

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/265938' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = 'proto_vision_xl_v66.safetensors' #@param {type:"string"}

#@markdown -----
#@markdown *추가익스텐션*

ControlNet = True 
ADetailer = True

#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = False #@param {type:"boolean"}

#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False #@param {type:"boolean"}

#@markdown **AnimateDiff 사용 (WebUI 버전 v1.6.0이상만 지원)**
AnimateDiff = False #@param {type:"boolean"}

#@markdown **mov2mov 사용**
Mov2mov = False #@param {type:"boolean"}

#@markdown **Deforum 사용**
Deforum = False #@param {type:"boolean"}

NotebookVersion = 'install_sdxl'
Workspace = 'drive/MyDrive/install'

if Google_Install:
  from google.colab import drive
  drive.mount('/content/drive')

if Google_Install:
  !mkdir -p /content/{Workspace}

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'

%cd /content

print('install...')
with capture.capture_output() as cap:
  !apt -y install -qq aria2
  !wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_{NotebookVersion}.sh -O /content/install.sh
  !chmod 755 /content/install.sh
  !./install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything} {AnimateDiff} {Mov2mov} {Deforum}
  !rm install.sh

%cd /content/{Workspace}

# 추가 익스텐션
# git clone https://github.com/vladmandic/sd-extension-system-info ./extensions/sd-extension-system-info

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' /content/{Workspace}/config.json

print('download...')
with capture.capture_output() as cap:
  # upscaler
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/ESRGAN -o 4x-UltraSharp.pth

  # checkpoint
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

  # vae
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt

  # lora
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors" -d /content/{Workspace}/models/Lora -o lcm-lora-sdxl.safetensors

  # embeddings
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/embeddings -o EasyNegative.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/embeddings -o badhandv4.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o negative_hand.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o bad_prompt_version2.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/5637?type=Model&format=PickleTensor&size=full&fp=fp16" -d /content/{Workspace}/embeddings/ -o ng_deepnegative_v1_75t.pt

!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/direct/{TagVersion}/directui.py -O /content/{Workspace}/directui.py
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/direct/{TagVersion}/launch_utils.py -O /content/{Workspace}/modules/launch_utils.py

print("------------------------------------------------------------------------------------------------")
print("설치완료!")
print("https://github.com/ninjaneural/webui/blob/master/memo/install.md 이 페이지를 참고해서 실행해주세요")
